In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score, cross_val_predict
from sklearn.metrics import (
    get_scorer, accuracy_score, recall_score, precision_score,
    roc_auc_score, matthews_corrcoef, average_precision_score
)
from tqdm import tqdm
import pandas as pd
import numpy as np
import scipy.stats
import copy

class Scrambler:
    def __init__(self, model, iterations=100):
        """
        初始化Scrambler类。

        :param model: 使用的模型
        :param iterations: 扰乱次数
        """
        self.base_model = model
        self.iterations = iterations
        self.progress_bar = False

    def validate(self, X, Y, method="train_test_split", scoring="accuracy", cross_val_score_aggregator="mean", pvalue_threshold=0.05, cv_kfolds=5, as_df=False, validation_data=None, progress_bar=False):
        """
        验证模型的稳健性。

        :param X: 特征数据
        :param Y: 目标数据
        :param method: 验证方法（train_test_split或cross_validation）
        :param scoring: 使用的评分方法
        :param cross_val_score_aggregator: 交叉验证的聚合方法
        :param pvalue_threshold: 显著性水平
        :param cv_kfolds: 交叉验证的折数
        :param as_df: 是否以DataFrame形式返回结果
        :param validation_data: 验证数据（如果有）
        :param progress_bar: 是否显示进度条
        :return: 根据as_df参数返回不同格式的评分结果
        """
        model_scorer = get_scorer(scoring)
        if model_scorer is None:
            raise Exception(f"scoring function '{scoring}' is not a valid sklearn scorer")

        if method == "train_test_split":
            result = self.__validate_train_test_split(X, Y, model_scorer, progress_bar)
        elif method == "cross_validation":
            result = self.__validate_cross_validation(X, Y, model_scorer, cross_val_score_aggregator, cv_kfolds, progress_bar)

        if as_df:
            return pd.DataFrame(result)
        else:
            return result

    def __validate_train_test_split(self, X, Y, scorer, progress_bar):
        """
        使用训练测试分割进行模型验证。

        :param X: 特征数据
        :param Y: 目标数据
        :param scorer: 评分函数
        :param progress_bar: 是否显示进度条
        :return: 各项评分的结果
        """
        X_train, X_test, Y_train, Y_test = train_test_split(X, Y)
        return self.__evaluate_model(X_train, Y_train, X_test, Y_test, scorer, progress_bar)

    def __validate_cross_validation(self, X, Y, scorer, aggregation, cv_kfolds, progress_bar):
        """
        使用交叉验证进行模型验证。

        :param X: 特征数据
        :param Y: 目标数据
        :param scorer: 评分函数
        :param aggregation: 聚合方法
        :param cv_kfolds: 折数
        :param progress_bar: 是否显示进度条
        :return: 各项评分的结果
        """
        return self.__evaluate_model(X, Y, X, Y, scorer, progress_bar, True, cv_kfolds)

    def __evaluate_model(self, X_train, Y_train, X_test, Y_test, scorer, progress_bar, cross_val=False, cv_kfolds=5):
        """
        评估模型并计算各项指标。

        :param X_train: 训练特征数据
        :param Y_train: 训练目标数据
        :param X_test: 测试特征数据
        :param Y_test: 测试目标数据
        :param scorer: 评分函数
        :param progress_bar: 是否显示进度条
        :param cross_val: 是否使用交叉验证
        :param cv_kfolds: 交叉验证的折数
        :return: 各项评分的结果
        """
        self.base_model.fit(X_train, Y_train)

        # 初始化各项评分列表
        accuracy_scores, recall_scores, precision_scores = [], [], []
        roc_auc_scores, mcc_scores, avg_precision_scores = [], [], []

        # 评估原始模型
        Y_pred = self.base_model.predict(X_test)
        accuracy_scores.append(accuracy_score(Y_test, Y_pred))
        recall_scores.append(recall_score(Y_test, Y_pred, average='binary'))
        precision_scores.append(precision_score(Y_test, Y_pred, average='binary'))
        roc_auc_scores.append(roc_auc_score(Y_test, Y_pred))
        mcc_scores.append(matthews_corrcoef(Y_test, Y_pred))
        avg_precision_scores.append(average_precision_score(Y_test, Y_pred))

        # 迭代并评估扰乱后的模型
        for _ in tqdm(range(self.iterations), disable=not progress_bar):
            Y_train_scrambled = np.random.permutation(Y_train)
            self.base_model.fit(X_train, Y_train_scrambled)
            Y_pred_scrambled = self.base_model.predict(X_test)
            accuracy_scores.append(accuracy_score(Y_test, Y_pred_scrambled))
            recall_scores.append(recall_score(Y_test, Y_pred_scrambled, average='binary'))
            precision_scores.append(precision_score(Y_test, Y_pred_scrambled, average='binary'))
            roc_auc_scores.append(roc_auc_score(Y_test, Y_pred_scrambled))
            mcc_scores.append(matthews_corrcoef(Y_test, Y_pred_scrambled))
            avg_precision_scores.append(average_precision_score(Y_test, Y_pred_scrambled))

        return {
            "accuracy_scores": accuracy_scores,
            "recall_scores": recall_scores,
            "precision_scores": precision_scores,
            "roc_auc_scores": roc_auc_scores,
            "mcc_scores": mcc_scores,
            "avg_precision_scores": avg_precision_scores
        }
